In [1]:
import cv2
import torch
import argparse
import os
import numpy as np
import json
import sys
sys.path.append("../")

from models import AttributeNet
from utils import load_yaml, read_img
from tqdm import tqdm
import random

In [2]:
def Path_Image_Preprocessing(path):
    '''
    Precessing the input images
        image_dir: single image input path, such as "/home/xxx/10.jpg"
    '''
    mean_bgr = np.array([91.4953, 103.8827, 131.0912])
    image = cv2.imread(path)
    if image is None:
        return None
    image = cv2.resize(image,(224,224))
    image = image.astype(np.float32)
    image -= mean_bgr
    # H * W * C   -->   C * H * W
    image = image.transpose(2,0,1)
    image = torch.tensor(image)
    image = image.unsqueeze(0)
    return image

In [4]:
device = "cuda:0"
model = AttributeNet() 

model.set_idx_list(attribute=["Male","Female","Young","Middle Aged","Senior","Asian","White","Black"])
model.to(device)
model.eval()

Load parameter conv1.weight
Load parameter bn1.weight
Load parameter bn1.bias
Load parameter bn1.running_mean
Load parameter bn1.running_var
Load parameter bn1.num_batches_tracked
Load parameter layer1.0.conv1.weight
Load parameter layer1.0.bn1.weight
Load parameter layer1.0.bn1.bias
Load parameter layer1.0.bn1.running_mean
Load parameter layer1.0.bn1.running_var
Load parameter layer1.0.bn1.num_batches_tracked
Load parameter layer1.0.conv2.weight
Load parameter layer1.0.bn2.weight
Load parameter layer1.0.bn2.bias
Load parameter layer1.0.bn2.running_mean
Load parameter layer1.0.bn2.running_var
Load parameter layer1.0.bn2.num_batches_tracked
Load parameter layer1.0.conv3.weight
Load parameter layer1.0.bn3.weight
Load parameter layer1.0.bn3.bias
Load parameter layer1.0.bn3.running_mean
Load parameter layer1.0.bn3.running_var
Load parameter layer1.0.bn3.num_batches_tracked
Load parameter layer1.0.downsample.0.weight
Load parameter layer1.0.downsample.1.weight
Load parameter layer1.0.downsa

AttributeNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [16]:
for i in range(1,10):
    path1 = os.path.join("image-4.4/real", str(i)+".jpg")
    path2 = os.path.join("image-4.4/fake", str(i)+".jpg")
    
    image_1 = Path_Image_Preprocessing(path1)
    image_2 = Path_Image_Preprocessing(path2)
    
    pred1 = model(image_1.to(device))[0]
    pred2 = model(image_2.to(device))[0]
    
    des1 = "Attribute: "
    if pred1[0] > 0.5:
        des1 += "Male, "
    elif pred1[1] > 0.5:
        des1 += "Female, "
        
    if pred1[2] > 0.5:
        des1 += "Young, "
    elif pred1[3] > 0.5:
        des1 += "Middle Aged, "
    elif pred1[4] > 0.5:
        des1 += "Old, "
    else:
        des1 += "Uncertain Age, "
        
    if pred1[5] > 0.5:
        des1 += "Asian."
    elif pred1[6] > 0.5:
        des1 += "White."
    elif pred1[7] > 0.5:
        des1 += "Black."
    else:
        des1 += "Uncertain Race."
    
    des2 = "Attribute: "
    if pred2[0] > 0.5:
        des2 += "Male, "
    elif pred2[1] > 0.5:
        des2 += "Female, "
        
    if pred2[2] > 0.5:
        des2 += "Young, "
    elif pred2[3] > 0.5:
        des2 += "Middle Aged, "
    elif pred2[4] > 0.5:
        des2 += "Old, "
    else:
        des2 += "Uncertain Age, "
        
    if pred2[5] > 0.5:
        des2 += "Asian."
    elif pred2[6] > 0.5:
        des2 += "White."
    elif pred2[7] > 0.5:
        des2 += "Black."
    else:
        des2 += "Uncertain Race."
    
    print("Image {}, left: {}, right: {}".format(i, des1, des2))

Image 1, left: Attribute: Female, Young, White., right: Attribute: Male, Young, White.
Image 2, left: Attribute: Female, Young, White., right: Attribute: Female, Young, Black.
Image 3, left: Attribute: Male, Old, White., right: Attribute: Male, Young, White.
Image 4, left: Attribute: Female, Young, White., right: Attribute: Male, Middle Aged, Black.
Image 5, left: Attribute: Male, Young, White., right: Attribute: Female, Middle Aged, White.
Image 6, left: Attribute: Female, Young, White., right: Attribute: Male, Middle Aged, Black.
Image 7, left: Attribute: Male, Young, Asian., right: Attribute: Female, Middle Aged, Asian.
Image 8, left: Attribute: Female, Young, White., right: Attribute: Male, Uncertain Age, Asian.
Image 9, left: Attribute: Female, Young, White., right: Attribute: Male, Middle Aged, Black.


tensor([1.3607e-03, 9.9864e-01, 9.9916e-01, 3.7283e-04, 4.6267e-04, 8.1762e-04,
        9.9900e-01, 1.7899e-04], device='cuda:0', grad_fn=<SelectBackward>)